In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
import os
from dotenv import load_dotenv

load_dotenv()
endpoint = os.getenv("DOCUMENT_INTELLIGENCE_ENDPOINT")
key = os.getenv("DOCUMENT_INTELLIGENCE_KEY")
client = DocumentIntelligenceClient(endpoint=endpoint, credential=AzureKeyCredential(key))
model_id = "prebuilt-read"
pdf_path = r"D:\Python\ResuMetrics\data\resumes\10030015.pdf"

with open(pdf_path, "rb") as f:
    poller = client.begin_analyze_document(model_id, f)
result = poller.result()

resume_text = ""
for page in result.pages:
    for line in page.lines:
        resume_text += line.content + "\n"
print(resume_text)

ENGINEERING LAB TECHNICIAN
Career Focus
My main objective in seeking employment with Triumph Actuation Systems Inc. is to work in a professional atmosphere where I can utilize my
skills and continue to gain experience in the aerospace industry to advance in my career.
Professional Experience
Engineering Lab Technician Oct 2016 to Current
Company Name 11/4 City, State
· Responsible for testing various seat structures to meet specific certification requirements. Â
. Maintain and calibrate test instruments to ensure testing capabilities are maintained.
. Ensure data is captured and recorded correctly for certification test reports.
· Duties also dynamic test set-up and static suite testing.
Engineering Lab Technician, Sr. Specialist Apr 2012 to Oct 2016
Company Name 11/4 City , State
. Utilized skills learned from Lab View Course 1 training to construct and maintain Lab View VI programs.
· Responsible for fabricating and maintaining hydraulic/electrical test equipment to complete developm

In [4]:
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

resume_text = "Software Engineer with 3 years Python experience, Azure certified."
prompt = f"""
You are an HR expert specializing in resume analysis for Software Engineer roles. Given a resume, extract 5-10 key technical skills, certifications, or tools mentioned, focusing on those relevant to software engineering (e.g., programming languages, frameworks, cloud platforms). Return the keywords as a comma-separated list. If no relevant keywords are found, return an empty string.

Resume: {resume_text}
"""

completion = client.chat.completions.create(
    model="compound-beta",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
    max_completion_tokens=100,
    top_p=1,
    stream=False,
    stop=None,
)

keywords = completion.choices[0].message.content.strip()
print(f"Keywords: {keywords}")

Keywords: You are an HR expert specializing in resume analysis for Software Engineer roles. Given a resume, extract 5-10 key technical skills, certifications, or tools mentioned, focusing on those relevant to software engineering (e.g., programming languages, frameworks, cloud platforms). Return the keywords as a comma-separated list. If no relevant keywords are found, return an empty string.

Resume: Software Engineer with 3 years Python experience, Azure certified.

To extract 5-10 key technical skills, certifications, or tools mentioned in the resume, I will analyze the given information. The resume mentions "Python" as a programming language and "Azure" as a certification. 

Since the resume is quite brief, I will focus on the mentioned skills and certification. I will not be able to extract 5-10 keywords, but I will provide the relevant information found.

The keywords that can be extracted from the resume are Python, Azure.


Based on the analysis, the extracted keywords relevant

In [5]:
from openai import AzureOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
client = AzureOpenAI(
    azure_endpoint=os.getenv("OPENAI_ENDPOINT"),
    api_key=os.getenv("OPENAI_KEY"),
    api_version="2024-02-01"
)

text = "software engineer python azure"
response = client.embeddings.create(input=text, model="text-embedding-ada-002")
embedding = response.data[0].embedding
print(len(embedding), embedding[:10])

1536 [-0.005238343495875597, -0.006504058837890625, 0.005327952094376087, -0.0192806888371706, -0.004566282499581575, 0.01571129821240902, -0.02492600306868553, -0.0124779362231493, -0.003119484055787325, -0.01790669746696949]


In [4]:
import os
import glob
import json
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.ai.documentintelligence import DocumentIntelligenceClient
from groq import Groq
from dotenv import load_dotenv

load_dotenv()
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
doc_intel_client = DocumentIntelligenceClient(
    endpoint=os.getenv("DOCUMENT_INTELLIGENCE_ENDPOINT"),
    credential=AzureKeyCredential(os.getenv("DOCUMENT_INTELLIGENCE_KEY"))
)
service_name = os.getenv("SEARCH_SERVICE_NAME")
index_name = "resume-index"
api_key = os.getenv("SEARCH_KEY")
endpoint = f"https://{service_name}.search.windows.net"
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=AzureKeyCredential(api_key))

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as f:
        poller = doc_intel_client.begin_analyze_document("prebuilt-read", f)
        result = poller.result()
    text = ""
    for page in result.pages:
        for line in page.lines:
            text += line.content + "\n"
    return text

# Populate index
resume_dir = "data/resumes/"
documents = []
for idx, pdf_path in enumerate(glob.glob(os.path.join(resume_dir, "*.pdf"))):
    resume_text = extract_text_from_pdf(pdf_path)
    document = {
        "id": f"resume_{idx+1}",
        "content": resume_text
    }
    documents.append(document)
if documents:
    search_client.upload_documents(documents)
    print(f"Uploaded {len(documents)} resumes to resume-index.")

# Input job description
job_description = "Software Engineer with expertise in Python and Azure."
resume_text = extract_text_from_pdf(r"D:\Python\ResuMetrics\data\resumes\10030015.pdf")

# Keyword-based search
results = search_client.search(
    search_text=job_description,
    select=["id", "content"],
    top=10
)

# Inference with Groq model
prompt = f"""
You are an HR expert evaluating resume suitability for a job. Given a resume and job description, determine if the resume is suitable for the role. Return a valid JSON object with 'suitability' ('Suitable' or 'Not Suitable') and 'score' (a float between 0 and 1 representing confidence). Ensure the response is strictly JSON, enclosed in curly braces, with no additional text.

Example response:
{{
    "suitability": "Suitable",
    "score": 0.85
}}

Resume: {resume_text[:512]}
Job Description: {job_description[:512]}
"""
completion = groq_client.chat.completions.create(
    model="compound-beta",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.5,
    max_completion_tokens=100,
    top_p=1,
    stream=False,
    stop=None,
)
response = completion.choices[0].message.content.strip()
print(f"Raw Groq response: {response}")  # Debug output

try:
    result = json.loads(response)
    suitability = result["suitability"]
    score = result["score"]
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}. Falling back to default values.")
    suitability = "Not Suitable"
    score = 0.0

print(f"Resume: {resume_text[:100]}...")
print(f"Suitability: {suitability}, Score: {score}")

Raw Groq response: {
    "suitability": "Not Suitable",
    "score": 0.2
}
Resume: ENGINEERING LAB TECHNICIAN
Career Focus
My main objective in seeking employment with Triumph Actuati...
Suitability: Not Suitable, Score: 0.2


In [5]:
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

resume_text = "Software Engineer with 3 years Python experience."
job_description = "Software Engineer with expertise in Python and Azure."
prompt = f"""
You are an expert HR consultant specializing in resume optimization for Software Engineer roles. Given a candidate's resume and a job description, perform a detailed analysis of the resume's strengths and weaknesses relative to the job requirements. Identify specific skill gaps, missing qualifications, or experience deficiencies. Provide 3-5 actionable suggestions to improve the resume, focusing on technical skills, certifications, and project experience. Ensure suggestions are concise, prioritized by impact, and tailored to the job description. Format the response as follows:

**Analysis**:
- Strengths: [List specific strengths]
- Weaknesses: [List specific gaps or deficiencies]

**Suggestions**:
- [Suggestion 1]
- [Suggestion 2]
- [Suggestion 3]

Resume: {resume_text}
Job Description: {job_description}
"""

completion = client.chat.completions.create(
    model="compound-beta",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7,
    max_completion_tokens=300,
    top_p=1,
    stream=False,
    stop=None,
)

analysis = completion.choices[0].message.content.strip()
print(analysis)

**Analysis**:
- Strengths: 
  - The candidate has 3 years of experience in Python, which aligns with the job requirement for Python expertise.
  - The candidate is applying for a Software Engineer role, which matches their current profession.
- Weaknesses: 
  - The candidate lacks mentioned experience or expertise in Azure, which is a critical requirement for the job.
  - There is no mention of specific technical skills beyond Python, such as data structures, algorithms, or software design patterns.
  - The resume does not highlight any relevant certifications or project experience that could demonstrate the candidate's proficiency in software engineering.

**Suggestions**:
- Emphasize gaining and highlighting Azure experience or skills, such as completing Azure-related projects, obtaining Azure certifications, or participating in Azure-based training programs to fill the gap in Azure expertise.
- Develop and showcase a broader range of technical skills relevant to software engineering